In [1]:
import cv2
import datetime
import threading

def capture_video(camera_index, output_file, stop_event):
    # Open a connection to the webcam
    cap = cv2.VideoCapture(camera_index)

    # Set the resolution to 1280x720 (1K)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # Define the codec and create VideoWriter object for MP4
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, 30.0, (1280, 720))

    if not cap.isOpened():
        print(f"Error: Could not open webcam {camera_index}.")
        return
    if not out.isOpened():
        print(f"Error: Could not open video writer for {output_file}.")
        return

    while not stop_event.is_set() and cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Get the current time
            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

            # Put the timestamp on the frame
            cv2.putText(frame, timestamp, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # Write the frame to the output file
            out.write(frame)

            # Display the resulting frame
            cv2.imshow(f'Camera {camera_index}', frame)

            # Break the loop on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                stop_event.set()
                break
        else:
            print(f"Error: Could not read frame from camera {camera_index}.")
            break

    # Release everything when done
    cap.release()
    out.release()
    cv2.destroyWindow(f'Camera {camera_index}')

# Create a stop event
stop_event = threading.Event()

# Create threads for each camera
threads = []
for i in range(1, 3):  # Use camera indices 1, 2, and 3
    output_file = f'output_cam_{i}.mp4'
    thread = threading.Thread(target=capture_video, args=(i, output_file, stop_event))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Destroy all windows
cv2.destroyAllWindows()